# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database into DataFrame df
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('DisasterData', con=engine)
# split Dataframe df into X and y
X = df['message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):

    # tokenize text and instantiate lemmatizer
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # remove punctuation
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    # remove stopwords
    tokens = [token for token in tokens if token not in stopwords.words('english')]

    # create clean tokens
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# build pipeline for the text transformation and for estimator
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# split data into training and test data, in this case into 70% training data and 30% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.3)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
%%time
# train data
pipeline.fit(X_train, y_train)

CPU times: user 57.7 s, sys: 4.06 s, total: 1min 1s
Wall time: 1min 2s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# create the classification_report
# I used this github repo as a reference https://github.com/beatboxerish/DSND/blob/master/Disaster_Reponse_Pipelines/disaster_response_pipeline_project/models/ML%20Pipeline%20Preparation.ipynb
def classification_rep_col(y_test, y_pred):
    for i, col in enumerate(y_test.columns): 
        print(col,': ',classification_report(y_test.iloc[:,i], y_pred[:,i]))

In [8]:
# predict responses for basic model
Y_pred = pipeline.predict(X_test)

In [9]:
# call classification_rep_col function
classification_rep_col(y_test, Y_pred)

related :               precision    recall  f1-score   support

          0       0.62      0.37      0.47      4343
          1       0.83      0.93      0.87     14008

avg / total       0.78      0.80      0.78     18351

request :               precision    recall  f1-score   support

          0       0.88      0.98      0.93     15172
          1       0.79      0.38      0.51      3179

avg / total       0.87      0.87      0.86     18351

offer :               precision    recall  f1-score   support

          0       1.00      1.00      1.00     18265
          1       0.00      0.00      0.00        86

avg / total       0.99      1.00      0.99     18351

aid_related :               precision    recall  f1-score   support

          0       0.73      0.85      0.79     10719
          1       0.72      0.57      0.64      7632

avg / total       0.73      0.73      0.72     18351

medical_help :               precision    recall  f1-score   support

          0       0.92  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# build pipeline for the text transformation and for estimator
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [11]:
# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 80, num = 10)]

In [12]:
# reduced hyperparameter tuning due to runtime
parameters = {
    'clf__estimator__n_estimators': n_estimators,
}

cv_forest = GridSearchCV(pipeline, param_grid = parameters, cv = 3, verbose = 1, n_jobs = 4)

In [13]:
%%time
# train improved model
cv_forest.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed: 63.6min finished


CPU times: user 2min 46s, sys: 4.75 s, total: 2min 51s
Wall time: 1h 6min 26s


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'clf__estimator__n_estimators': [20, 26, 33, 40, 46, 53, 60, 66, 73, 80]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [14]:
# use the best performing parameters
cv_forest.best_params_

{'clf__estimator__n_estimators': 80}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
# predict responses for improved model
Y_pred_cv = cv_forest.predict(X_test)

In [16]:
# call classification_rep_col function
classification_rep_col(y_test, Y_pred_cv)

related :               precision    recall  f1-score   support

          0       0.71      0.27      0.40      4343
          1       0.81      0.97      0.88     14008

avg / total       0.79      0.80      0.77     18351

request :               precision    recall  f1-score   support

          0       0.90      0.98      0.94     15172
          1       0.82      0.46      0.59      3179

avg / total       0.88      0.89      0.88     18351

offer :               precision    recall  f1-score   support

          0       1.00      1.00      1.00     18265
          1       0.00      0.00      0.00        86

avg / total       0.99      1.00      0.99     18351

aid_related :               precision    recall  f1-score   support

          0       0.78      0.85      0.81     10719
          1       0.75      0.66      0.71      7632

avg / total       0.77      0.77      0.77     18351

medical_help :               precision    recall  f1-score   support

          0       0.92  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

I'm going to use the KNeighborsClassifier like in my previous submission and see how well it performs in this case.

In [17]:
# using KNeighborsClassifier
pipeline_knn = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

In [18]:
# train KNeighborsClassifier model
pipeline_knn.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

In [19]:
# predict responses for KNeighborsClassifier model
Y_pred_knn = pipeline_knn.predict(X_test)

In [20]:
# call classification_rep_col function
classification_rep_col(y_test, Y_pred_knn)

related :               precision    recall  f1-score   support

          0       0.58      0.38      0.46      4343
          1       0.83      0.91      0.87     14008

avg / total       0.77      0.79      0.77     18351

request :               precision    recall  f1-score   support

          0       0.90      0.93      0.92     15172
          1       0.62      0.53      0.57      3179

avg / total       0.86      0.86      0.86     18351

offer :               precision    recall  f1-score   support

          0       1.00      1.00      1.00     18265
          1       0.00      0.00      0.00        86

avg / total       0.99      1.00      0.99     18351

aid_related :               precision    recall  f1-score   support

          0       0.73      0.75      0.74     10719
          1       0.63      0.60      0.62      7632

avg / total       0.69      0.69      0.69     18351

medical_help :               precision    recall  f1-score   support

          0       0.93  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [21]:
saved_model = pickle.dumps(cv_forest.best_estimator_)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.